In [62]:
import torch
import json
import numpy as np
import torchvision

from PIL import Image
from utility import ViltImageSetProcessor
from transformers import ViltImageProcessor, BertTokenizer, ViTModel, ViTConfig, ViTImageProcessor
from models import MultiviewViltForQuestionAnswering, MultiviewViltModel
from torch import nn
from copy import deepcopy
from isvqa_data_setup import ISVQA
from torch.utils.data import DataLoader
from engine import max_to_one_hot
from collections import Counter
from torch.optim.lr_scheduler import StepLR
from torchvision import transforms

In [14]:
import json
import random

from copy import deepcopy

with open("/home/nikostheodoridis/nuscenes/v1.0-trainval/sample_data.json") as f:
    set = json.load(f)


In [2]:
weights = torchvision.models.ViT_B_16_Weights
vit = torchvision.models.vit_b_16(weights)

/home/nikostheodoridis/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/nikostheodoridis/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /home/nikostheodoridis/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:02<00:00, 116MB/s]  


In [45]:
vit2 = ViTModel(ViTConfig()).to("cuda")

In [74]:
vit2 = ViTModel(ViTConfig())
img = torch.randn((1, 3, 224, 224))

out = vit2(img)
my_pooler = torch.matmul(vit2.pooler.dense.weight, out[0][:, 0].T).T + vit2.pooler.dense.bias

In [75]:
out

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-1.9228, -0.0363, -1.3237,  ...,  0.6033,  0.4010, -0.2963],
         [ 0.2995,  0.2434, -0.3559,  ...,  1.2787, -0.2542, -1.2179],
         [ 0.1287,  0.5816, -0.2105,  ..., -0.1496,  0.2117, -0.6212],
         ...,
         [-2.0719,  0.7310,  0.1939,  ...,  1.0240,  1.2290,  0.9660],
         [ 0.6128, -0.6037, -0.4326,  ...,  0.3306, -1.2665,  0.9593],
         [-1.5866,  0.2553,  0.0570,  ...,  0.3944, -0.8897,  1.5030]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-1.2910e-01,  4.7080e-01, -2.3381e-01, -1.7056e-01,  4.1828e-01,
          1.3459e-01,  5.4096e-01,  7.7265e-01,  6.4455e-01,  1.5180e-01,
          1.5184e-01, -2.4485e-01,  3.9936e-01,  2.3162e-01,  3.9372e-01,
          7.9987e-02,  2.7627e-01,  6.1587e-01, -2.1772e-01,  3.6806e-01,
         -5.2010e-01, -7.8098e-01,  3.4164e-01,  4.2666e-01, -7.6649e-01,
         -2.7599e-01,  7.1899e-01, -4.6672e-01, -6.8824e-01,  3.1623e-01,
          7

In [31]:
out[1].shape, out[0][:, 0].shape

(torch.Size([1, 768]), torch.Size([1, 768]))

In [51]:
my_pooler = torch.matmul(vit2.pooler.dense.weight, out[0][:, 0].T).T + vit2.pooler.dense.bias

tanh = nn.Tanh()

my_pooler = tanh(my_pooler)

In [41]:
my_pooler.shape

torch.Size([1, 768])

In [53]:
torch.equal(my_pooler, out[1])

True

In [48]:
print(vit2)

ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation(

In [59]:
processor = ViTImageProcessor()

In [61]:
img = Image.open("/home/nikostheodoridis/nuscenes/samples/CAM_BACK/n008-2018-05-21-11-06-59-0400__CAM_BACK__1526915243037570.jpg")

In [63]:
tensor_img = transforms.ToTensor()(img)

In [67]:
processor(tensor_img, do_rescale=False, return_tensors="pt")["pixel_values"].shape

torch.Size([1, 3, 224, 224])

In [72]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/home/nikostheodoridis/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [73]:
count_parameters(model)

Modules,Parameters
embeddings.word_embeddings.weight,23440896
embeddings.position_embeddings.weight,393216
embeddings.token_type_embeddings.weight,1536
embeddings.LayerNorm.weight,768
embeddings.LayerNorm.bias,768
encoder.layer.0.attention.self.query.weight,589824
encoder.layer.0.attention.self.query.bias,768
encoder.layer.0.attention.self.key.weight,589824
encoder.layer.0.attention.self.key.bias,768
encoder.layer.0.attention.self.value.weight,589824


In [12]:
scheduler_type = None


if scheduler_type == "steplr":
        print(1)
elif scheduler_type is not None:
    print(2)
else:
    print(3)

3


In [6]:
optimizer = torch.optim.Adam(model.parameters())

In [8]:
scheduler = StepLR(optimizer=optimizer, step_size=None, gamma=None)

In [56]:
isvqa[0][0]["pixel_values"].shape

torch.Size([6, 3, 352, 608])

In [54]:
isvqa = ISVQA("/home/nikostheodoridis/isvqa/train_set.json",
              "/home/nikostheodoridis/nuscenes/samples",
              "/home/nikostheodoridis/isvqa/answers.json")

/home/nikostheodoridis/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [40]:
model = MultiviewViltForQuestionAnswering(6, 210, 40, 768, True, True, True).to("cpu")
layer = nn.TransformerEncoderLayer(768, 12, batch_first=True, dim_feedforward=3072)
# model = nn.TransformerEncoder(layer, 12)

/home/nikostheodoridis/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params += param
    table.add_row(["Total", total_params])
    return table



In [4]:
loader = DataLoader(isvqa, 4, shuffle=False)
batch = next(iter(loader))

In [5]:
model(**batch[0])

SequenceClassifierOutput(loss=None, logits=tensor([[ -9.9990, -11.7384,  -7.4223,  ...,  -9.9704, -11.2511, -10.7394],
        [ -9.6335, -11.7417,  -7.1838,  ..., -10.0912, -11.4883, -10.6841],
        [ -9.4269, -10.0896,  -7.3474,  ..., -11.3587, -12.3218,  -9.9821],
        [ -9.4646, -10.1867,  -7.3609,  ..., -11.3594, -12.3701, -10.1991]],
       device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [32]:
questions = torch.randn(4, 1, 768)
images = torch.randn(4, 6, 768)

attn_scores = torch.randn(4, 1, 6)

In [ ]:
attn = 

In [ ]:
weight = torch.randn()


In [ ]:
import torch
from prettytable import PrettyTable

def count_model_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params += param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [ ]:
loader = DataLoader(isvqa, 1, shuffle=False)

In [ ]:
batch = next(iter(loader))

In [ ]:
out = model(**batch[0])

In [ ]:
out.last_hidden_state.shape

AttributeError: 'torch.Size' object has no attribute 'last_hidden_state'

In [42]:
layer = nn.TransformerEncoderLayer(768, 12, batch_first=True, dim_feedforward=3072)

src = torch.randn(4, 6, 768)

out = layer(src)

In [43]:
layer = nn.TransformerEncoderLayer(768, 12, batch_first=True, dim_feedforward=3072)

questions = torch.randn(4, 1, 768)
images = torch.randn(4, 6, 768)

img_attn = nn.MultiheadAttention(768, 12, batch_first=True)

_, attn_scores = img_attn(questions, images, images)

out = layer(images)

In [46]:
out.shape

torch.Size([4, 6, 768])

In [53]:
decoder_layer = nn.TransformerDecoderLayer(d_model=768, nhead=12, batch_first=True)
questions = torch.randn(4, 1, 768)
images = torch.randn(4, 6, 768)

out = decoder_layer(questions, images)
out2 = decoder_layer(images, questions)

In [51]:
out.shape

torch.Size([4, 1, 768])

In [54]:
out2.shape

torch.Size([4, 6, 768])

In [52]:
out[0]

tensor([[ 2.8202e-01, -4.3938e-01,  5.2543e-01,  1.3738e+00,  1.3679e+00,
          1.3698e+00,  1.6462e+00,  3.5957e-01, -1.0734e+00,  1.0604e+00,
          1.4331e-01,  7.0762e-01, -2.4905e-01, -1.6860e+00,  7.7606e-01,
         -4.8757e-01, -1.0267e+00,  1.4065e+00,  1.4866e-01, -8.0037e-01,
          1.6971e+00, -4.9145e-01,  6.6795e-01,  4.9078e-01,  5.7478e-02,
          4.1770e-01,  1.1652e+00,  1.3306e+00, -1.7638e+00,  2.7130e-01,
          1.8238e+00, -1.3587e+00, -7.1948e-01,  2.6626e-01,  1.2651e+00,
         -1.0523e+00,  1.3528e+00,  4.7452e-01, -3.1470e-02,  1.4822e+00,
          4.2859e-01,  7.7379e-01,  3.6558e-01,  7.0166e-01, -1.0859e+00,
          1.6979e+00, -2.1000e-01, -1.0874e+00, -1.4302e+00,  4.2423e-01,
         -2.3164e-01,  1.4025e+00, -1.2105e+00,  1.8429e+00, -5.3007e-01,
          1.7268e+00,  5.8075e-01,  1.9058e-02,  3.9585e-01,  1.2251e+00,
         -7.9854e-01,  4.8526e-01, -2.8563e-02,  1.4292e+00,  2.6455e-01,
         -8.8936e-01, -7.0890e-01, -1.

In [ ]:
def val_step(model, loader, acc_fn, answ_len):
    """
    A function that validates the model by going through all the mini-batches in the validation dataloader once.
    """
    print("\tValidating...")
    model.eval()
    losses = []  # to save the loss of each mini-batch in order to take their average at the end
    accuracies = []  # to save the accuracy of each mini-batch in order to take their average at the end

    predictions = []
    

    with torch.inference_mode():
        for i, (X, y) in enumerate(loader):
            outputs = model(**X, labels=y)
            loss = outputs.loss
            pred = max_to_one_hot(outputs.logits)
            acc = acc_fn(pred, y, answ_len)

            losses.append(loss.item())
            accuracies.append(acc)

    avg_loss = sum(losses) / len(loader)
    avg_acc = sum(accuracies) / len(loader)
    
    return avg_loss, avg_acc

# Copy

In [ ]:
model = MultiviewViltForQuestionAnswering(6, 210, 768, True, False, False).to("cuda")

/home/nikostheodoridis/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
model.model.classifier = nn.Sequential(
        nn.Linear(768, 1536),
        nn.LayerNorm(1536),
        nn.GELU(),
        nn.Linear(1536, 429)
    ).to("cuda")

In [ ]:
trained_model = deepcopy(model)

In [ ]:
trained_model.load_state_dict(torch.load("/home/nikostheodoridis/Trained Models/2024-07-08 00:07:49/model.pth"))

<All keys matched successfully>

In [ ]:
# for p1, p2 in zip(model.parameters(), trained_model.parameters()):
#     assert torch.equal(p1, p2)

In [ ]:
val_set = ISVQA(qa_path="/home/nikostheodoridis/isvqa/val_set.json",
                nuscenes_path="/home/nikostheodoridis/nuscenes/samples",
                answers_path="/home/nikostheodoridis/isvqa/answers.json")

In [ ]:
val_loader = DataLoader(val_set, batch_size=6, shuffle=False)

In [ ]:
# targets = []
# untrained_predictions = []
# trained_predictions = []

# model.eval()
# trained_model.eval()
# for i in range(2576):
#     inputs, target = val_set[i]

#     targets.append(target)

#     with torch.inference_mode():
        


In [ ]:
def accuracy(predictions: torch.Tensor, targets: torch.Tensor, answers_len: int) -> float:
    cnt = torch.eq(torch.eq(predictions, targets).sum(dim=1), answers_len).sum()
    return cnt.item() / len(predictions)

In [ ]:
untrained_loss, untrained_acc = val_step(model, val_loader, accuracy, 429)

	Validating...


In [ ]:
trained_loss, trained_acc = val_step(trained_model, val_loader, accuracy, 429)

	Validating...


In [ ]:
print(untrained_loss)
print(untrained_acc)

303.8099614342978
0.0011627906976744186


In [ ]:
print(trained_loss)
print(trained_acc)

2.5099486532945967
0.6003875968992246


In [ ]:
with open("/home/nikostheodoridis/isvqa/answers_counter.json") as f:
    answers_cnt = json.load(f)

In [ ]:
Counter(answers_cnt)

Counter({'yes': 13564,
         'no': 3734,
         'one': 3663,
         'white': 2893,
         'two': 2544,
         'red': 1205,
         'black': 1046,
         'blue': 1025,
         'three': 986,
         'green': 968,
         'yellow': 930,
         'orange': 782,
         'four': 529,
         'night': 464,
         'rainy': 434,
         'gray': 365,
         'black and white': 345,
         'silver': 287,
         'zero': 254,
         'five': 218,
         'orange and white': 178,
         'six': 156,
         'left': 151,
         'none': 147,
         'ahead': 147,
         'right': 141,
         'fedex': 136,
         'brown': 134,
         'cloudy': 129,
         'slow': 119,
         'ups': 114,
         'bus': 112,
         'raining': 111,
         'wet': 111,
         'sunny': 107,
         'ryder': 95,
         'urban': 93,
         'twenty-three': 92,
         'stop': 89,
         'day': 77,
         'hump': 69,
         'brick': 69,
         'red and white': 57,

In [ ]:
# Start
import json
import random
train_path = "/home/nikostheodoridis/nuscenes-qa/train_set.json"

val_path = "/home/nikostheodoridis/nuscenes-qa/val_set.json"

test_path = "/home/nikostheodoridis/nuscenes-qa/test_set.json"
with open(train_path) as f:
    train_data = json.load(f)

with open(val_path) as f:
    val_data = json.load(f)

with open(test_path) as f:
    test_data = json.load(f)

for data in train_data:
    if data in test_data:
        print("False")
        break
else:
    print("True")

True
